In [18]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np


In [70]:
api_key= ""

In [7]:
response = response = requests.get(
    "https://www.deeplearning.ai/the-batch/a-roadmap-explores-how-ai-can-detect-and-mitigate-greenhouse-gases/"
)
soup = BeautifulSoup(response.text, "html.parser")
tag = soup.find("div", re.compile("^prose--styled"))
text = tag.text
file_name = "AI_greenhouse_gas.txt"
with open(file_name, 'w') as file:
    file.write(text)

In [8]:
chunk_size=512
len(text)

4010

In [13]:
chunks = []
for i in range(0, len(text), chunk_size):
    chunks.append(text[i:i+chunk_size])

In [17]:
import os
from mistralai.client import MistralClient


def get_text_embeddings(txt:str):
    client = MistralClient(api_key=api_key)
    emb_res = client.embeddings(model="mistral-embed",input=txt)
    return emb_res.data[0].embedding

In [19]:
text_embeddings =  np.array([get_text_embeddings(txt) for txt in chunks])

In [22]:
text_embeddings.shape[1]

1024

In [24]:
question = "What are the ways that AI can reduce emissions in Agriculture?"
question_embeddings = np.array([get_text_embeddings(question)])

In [29]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [30]:
# index = faiss.IndexFlatL2(d)
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x114849110> >

In [32]:
D,I = index.search(question_embeddings, k=2)

In [33]:
print(D,I)

[[0.2616668  0.29929367]] [[4 5]]


In [41]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['data to help factories use more recycled materials, cut waste, minimize energy use, and reduce downtime. Similarly, they can optimize supply chains to reduce emissions contributed by logistics.\xa0Agriculture.\xa0Farmers use AI-equipped sensors to simulate different crop rotations and weather events to forecast crop yield or loss. Armed with this data, food producers can cut waste and reduce carbon footprints. The authors cite lack of food-related datasets and investment in adapting farming practices as primary b', 'arriers to taking full advantage of AI in the food industry.Transportation.\xa0AI systems can reduce greenhouse-gas emissions by improving traffic flow, ameliorating congestion, and optimizing public transportation. Moreover, reinforcement learning can reduce the impact of electric vehicles on the power grid by optimizing their charging. More data, uniform standards, and AI talent are needed to realize this potential.Materials.\xa0Materials scientists use AI models to stu

In [42]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [48]:
from mistralai.models.chat_completion import ChatMessage


def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [49]:
prompt

"\nContext information is below.\n---------------------\n['data to help factories use more recycled materials, cut waste, minimize energy use, and reduce downtime. Similarly, they can optimize supply chains to reduce emissions contributed by logistics.\\xa0Agriculture.\\xa0Farmers use AI-equipped sensors to simulate different crop rotations and weather events to forecast crop yield or loss. Armed with this data, food producers can cut waste and reduce carbon footprints. The authors cite lack of food-related datasets and investment in adapting farming practices as primary b', 'arriers to taking full advantage of AI in the food industry.Transportation.\\xa0AI systems can reduce greenhouse-gas emissions by improving traffic flow, ameliorating congestion, and optimizing public transportation. Moreover, reinforcement learning can reduce the impact of electric vehicles on the power grid by optimizing their charging. More data, uniform standards, and AI talent are needed to realize this poten

In [50]:
mistral(prompt)

'In Agriculture, AI-equipped sensors can be used by farmers to simulate different crop rotations and weather events, which helps in forecasting crop yield or loss. This data enables food producers to cut waste and reduce their carbon footprints.'

In [63]:
def qa_with_context(text, question, chunk_size=512):
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    text_embeddings = np.array([get_text_embeddings(chunk) for chunk in chunks])
    d = text_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(text_embeddings)
    question_embeddings = np.array([get_text_embeddings(question)])
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """
    response = mistral(prompt)
    return response
    

In [64]:
import functools

names_to_functions = {"qa_with_context": functools.partial(qa_with_context, text=text)}

In [65]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "qa_with_context",
            "description": "Answer user question by retrieving relevant context",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "user question",
                    }
                },
                "required": ["question"],
            },
        },
    },
]

In [66]:
question = """
What are the ways AI can mitigate climate change in transportation?
"""

client = MistralClient(api_key=api_key)

response = client.chat(
    model="mistral-large-latest",
    messages=[ChatMessage(role="user", content=question)],
    tools=tools,
    tool_choice="any",
)

response

ChatCompletionResponse(id='42c433a187b040b9ae2abb442fb4ad26', object='chat.completion', created=1714289352, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='n6wo2XjWc', type=<ToolType.function: 'function'>, function=FunctionCall(name='qa_with_context', arguments='{"question": "What are the ways AI can mitigate climate change in transportation?"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=92, total_tokens=126, completion_tokens=34))

In [67]:
tool_function = response.choices[0].message.tool_calls[0].function
tool_function

FunctionCall(name='qa_with_context', arguments='{"question": "What are the ways AI can mitigate climate change in transportation?"}')

In [68]:
import json

args = json.loads(tool_function.arguments)
args

{'question': 'What are the ways AI can mitigate climate change in transportation?'}

In [69]:
names_to_functions[tool_function.name](**args)

'The context information does not provide specific details on how AI can mitigate climate change in the transportation sector. However, it does mention that AI has potential in various sectors including manufacturing, food production, and transportation to make a significant dent in greenhouse gas emissions. For a more detailed answer, additional information or sources would be needed.'